In [ ]:
START

In [1]:
!pip -q install "transformers>=4.41" peft bitsandbytes accelerate datasets sentencepiece einops pydicom pillow

import os, re, shutil, math, random
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.8 MB/s eta 0:00:00


Mounted at /content/drive
exists: True


In [ ]:
####Mount the drive
################ attach the colab project to the google drive
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
ROOT        = Path('/content/drive/MyDrive/MLProjects/foundation-models-radiology')
DICOM_DIR   = ROOT / 'PTXHeadtoHeadSmall'   # use the exact folder name as on Drive
IMG_DIR     = ROOT / "chex_jpg"                   # where we'll write JPEGs
CSV_PATH    = ROOT / "labels_200.csv"             # your instruction CSV (image,question,answer)

print("exists:", DICOM_DIR.exists())


In [ ]:
################ Global variables
MODEL_NAME = "StanfordAIMI/CheXagent-2-3b"
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)



In [ ]:
#########DICOM to grayscale
shutil.rmtree(IMG_DIR, ignore_errors=True)
IMG_DIR.mkdir(parents=True, exist_ok=True)

def dicom_to_jpg(dcm_path: Path, out_dir: Path) -> Path:
    ds = pydicom.dcmread(str(dcm_path), stop_before_pixels=False)
    arr = ds.pixel_array

    # Respect VOI LUT (windowing) when present
    try:
        arr = apply_voi_lut(arr, ds)
    except Exception:
        pass

    # Fix MONOCHROME1 inversion
    if getattr(ds, "PhotometricInterpretation", "") == "MONOCHROME1":
        arr = np.max(arr) - arr

    arr = arr.astype(np.float32)
    arr -= arr.min()
    if arr.max() > 0:
        arr /= arr.max()
    arr = (arr * 255.0).clip(0,255).astype(np.uint8)

    # Optional: resize to something reasonable (e.g., 1024px max side)
    h, w = arr.shape[:2]
    max_side = 1024
    if max(h,w) > max_side:
        scale = max_side / max(h,w)
        arr = np.array(Image.fromarray(arr).resize((int(w*scale), int(h*scale)), Image.LANCZOS))

    img = Image.fromarray(arr, mode="L")
    out_path = out_dir / (dcm_path.stem + ".jpg")
    img.save(out_path, quality=95)
    return out_path

# find dicoms, convert first ~200
dcm_paths = list(DICOM_DIR.rglob("*.[dD][cC][mM]"))
dcm_paths = dcm_paths[:200] if len(dcm_paths) > 200 else dcm_paths
jpg_paths = [dicom_to_jpg(p, IMG_DIR) for p in dcm_paths]
len(jpg_paths), jpg_paths[:3]


In [ ]:


# step 1: Setup constant
model_name = "StanfordAIMI/CheXagent-2-3b"
dtype = torch.bfloat16
device = "cuda"
